# Create multi-band raster with rasterio

This script allows to stack a list of single band raster into a unique multi-band raster.
The most common case is when you want to create a color composite. But it could also be useful to create a multi-band raster with all the images of a timeserie for example.

In [1]:
import glob, os
import numpy as np
import pandas as pd
import rasterio
from rasterio import plot
from rasterio.plot import show
from pathlib import Path

## Set paths for input and output directories

In [2]:
# Directory for all work files

grp_letter = 'X'
#work_path = f'X:/GROUP_{grp_letter}/WORK/'
work_path = f'/Volumes/nbdid-sst-lbrat2104/GROUP_{grp_letter}/WORK/'


multi_path = f'{work_path}MULTIBAND/'

Path(multi_path).mkdir(parents=True, exist_ok=True)

print(f'General work path is set to      : {work_path}')
print(f'Multiband rasters path is set to : {multi_path}')

General work path is set to      : /Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/
Multiband rasters path is set to : /Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/MULTIBAND/


## Get list of single band images to stack in a single multi-band GeoTIFF

### 1. True Color Composite - select bands RED / GREEN / BLUE

In [6]:
date = '20200212'

reflectance_path = f'{work_path}2_L2A_CLIPPED/'
multi_band_file = f'{multi_path}TrueColor_composite_{date}.tif'

red_band   = glob.glob(f'{reflectance_path}*{date}*B04*.tif')[0]
green_band = glob.glob(f'{reflectance_path}*{date}*B03*.tif')[0]
blue_band  = glob.glob(f'{reflectance_path}*{date}*B02*.tif')[0]

list_im = [red_band, green_band, blue_band]

### 2. False Color Composite - select bands RED / GREEN / NIR

In [4]:
date = '20200212'

reflectance_path = f'{work_path}3_L2A_MASKED/'
multi_band_file = f'{multi_path}FalseColor_composite_{date}.tif'


red_band   = glob.glob(f'{reflectance_path}*{date}*B04*.tif')[0]
green_band = glob.glob(f'{reflectance_path}*{date}*B03*.tif')[0]
nir_band   = glob.glob(f'{reflectance_path}*{date}*B08*.tif')[0]

list_im = [nir_band, red_band, green_band]

### 3. Timeserie - select all rasters in the period of the timeserie

In [5]:
index_path = f'{work_path}NDVI/'

multi_band_file = f'{multi_path}NDVI_timeserie.tif'

list_im = sorted(glob.glob(f'{index_path}*.tif'))

## Get metadata from one image of the list and update number of layers/bands

In [7]:
src = rasterio.open(list_im[0], 'r')

profile = src.profile
profile.update(count=len(list_im))

print(f'Number of layers : {profile["count"]}')

src.close()

Number of layers : 3


In [9]:

if not os.path.isfile(multi_band_file):

    list_src_arr = []

    for im_file in list_im:
        print(im_file)
        src = rasterio.open(im_file, 'r')
        im = src.read(1)
        list_src_arr.append(im)
        src.close()

    with rasterio.open(multi_band_file, 'w', **profile) as dst:
        for band_nr, src in enumerate(list_src_arr, start=1):
            dst.write(src, band_nr)
    
    print(f'--> A new mulit-band raster file is created : {multi_band_file}')

else:
    print(f'--> {multi_band_file} - already exists')


--> /Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/MULTIBAND/TrueColor_composite_20200212.tif - already exists
